In [1]:
from langchain_community.llms import Ollama
import pprint

In [2]:
AGENT_MODEL_NAME = "arcee-ai/arcee-agent"

In [3]:
agent = Ollama(
    model = AGENT_MODEL_NAME,
    system = """You are a helpful chat assistant""",
    top_p = 0.7,
)

In [4]:
func_prompt = """You have four primary functions: retrieving available packages, retrieving special offers, and retrieving new packages, and answering specific questions user about packages and promotions. Use the appropriate function based on the user's query.

### Functions:

1. **get_list_data() -> tuple**
    - This function sends a GET request to the '/list' endpoint to retrieve the list data.
    - Input: None.
    - Output: A JSON object containing the list of available packages from the server. Example categories include mobile broadband, home broadband, and dialog TV.

2. **get_special_offers() -> tuple**
    - This function sends a GET request to the '/special_offers' endpoint to retrieve the special offers.
    - Input: None.
    - Output: A plain text string containing the formatted special offers from the server. Example offers include home broadband offers, mobile broadband offers, and TV offers.

3. **get_new_packages_data() -> tuple**
    - This function sends a GET request to the '/new_package' endpoint to retrieve the new packages data.
    - Input: None.
    - Output: A JSON object containing the new packages data from the server. Example packages include new mobile plans and new home broadband plans.
    
4. **answer_general_question(question: str) -> str**
    - This function answers questions in general.
    - Input: a user question.
    - Output: your best answer to the user question.

### Instructions:

- If the user asks for a list of available packages (keywords: "available packages", "list of packages", "broadband plans", "TV packages"), call `get_list_data` function to answer.
- If the user asks for current special offers or promotions (keywords: "special offers", "promotions", "current deals"), use the `get_special_offers` function.
- If the user asks for information about new packages (keywords: "new packages", "new plans", "recently introduced packages"), use the `get_new_packages_data` function.
- If the user asks any other question, use the `answer_general_question` function. Only return the result about telecommunication.
- Only return the result of the function call, not your internal reasoning.

### User Query:\n"""

In [5]:
import requests
# def get_register_status(nic_number):
#     """
#     Send a POST request to the '/register_status' endpoint to check if a NIC number is registered.

#     Args:
#         nic_number (str): The NIC number to check.

#     Returns:
#         dict: The JSON response from the server.
#     """
#     url = 'http://localhost:5000/register_status'
#     data = {'nic_number': nic_number}
#     response = requests.post(url, data=data)
#     return response.json(), response.status_code

def get_list_data():
    """
    Send a GET request to the '/list' endpoint to retrieve the list data.

    Returns:
        dict: The JSON response from the server.
    """
    url = 'http://localhost:5000/list'
    response = requests.get(url)
    return response.json(), response.status_code

def get_special_offers():
    """
    Send a GET request to the '/special_offers' endpoint to retrieve the special offers.

    Returns:
        str: The plain text response from the server.
    """
    url = 'http://localhost:5000/special_offers'
    response = requests.get(url)
    return response.text, response.status_code

def get_new_packages_data():
    """
    Send a GET request to the '/new_package' endpoint to retrieve the new packages data.

    Returns:
        dict: The JSON response from the server.
    """
    url = 'http://localhost:5000/new_package'
    response = requests.get(url)
    return response.json(), response.status_code

def answer_general_question(question):
    ans = agent.invoke(f"""{question}\n""")
    return ans

In [6]:
def function_call(input, model=agent, func_prompt=func_prompt):
    """
    Interacts with an AI model by combining user input with a custom prompt.

    Args:
        input (str): The user's input string.
        model (object, optional): The AI model to invoke. Defaults to agent.
        func_prompt (str, optional): The custom prompt to combine with the user's input. Defaults to func_prompt.

    Returns:
        str: The response generated by the AI model in response to the combined prompt and input.
    """
    payload = f"""{func_prompt}{input}\n"""
    ans = model.invoke(payload)
    return ans

In [7]:
def llm_pack(input):
    """
    Evaluates a string generated by an AI model and returns the result.

    This function wraps around 'function_call' to convert the AI's response into a executable code snippet,
    executes it, and then returns the result. If any exception occurs during execution, it returns None.

    Args:
        input (str): The input to pass to 'function_call'.

    Returns:
        str: The result of the function call.
    """
    try:
        func = function_call(input)
        # func = func.strip()
        print(func)
        code = f"result = {func}"
        local_vars = {}
        exec(code, globals(), local_vars)
        ans = local_vars.get('result')
        return ans
    except Exception as e:
        print(f"Error processing LLM output: {e}")
        return None

In [14]:
response = llm_pack("what are available packages")
pprint.pprint(response)

get_list_data()
({'dialog_tv': [{'connectionType': 'Postpaid',
                 'packages': ['Pearl', 'Thee', 'Diamond']},
                {'connectionType': 'Prepaid',
                 'packages': ['Super Saver', 'Super Lite', 'Super Value']}],
  'home_broadband': [{'connectionName': 'Postpaid',
                      'packages': ['Anytime', 'Unlimited', 'Gaming']},
                     {'connectionName': 'Prepaid',
                      'packages': ['20GB Super saver 30 day add-on',
                                   'YouTube & Social Media plan',
                                   'Work & Learn Plus 672']}],
  'mobile_broadband': [{'connectionName': 'Postpaid'},
                       {'connectionName': 'Prepaid'}]},
 200)
